In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import re
import contractions
from bs4 import BeautifulSoup
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer,LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rupak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rupak\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rupak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\rupak\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [20]:
test = "I can't do this anymore 1 <head> ?? I am running away now"
test= contractions.fix(test)
text = re.sub(r'[^\w\s!?]', ' ', test)
text = re.sub(r'\d', '', test)
print(text)

I cannot do this anymore  <head> ?? I am running away now


In [39]:
lemmatizer = WordNetLemmatizer()


#function to map NLTK POS tags to WordNet POS tags
def get_wordnet_pos(nltk_pos):
    if nltk_pos.startswith('J'):  # Adjective
        return 'a'
    elif nltk_pos.startswith('V'):  # Verb
        return 'v'
    elif nltk_pos.startswith('N'):  # Noun
        return 'n'
    elif nltk_pos.startswith('R'):  # Adverb
        return 'r'
    else:
        return wordnet.NOUN  # Default to noun

#preprocessing function
def clean_data(data):
    #expand contractions
    text = contractions.fix(data)
    
    #remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    
    #remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    #remove numbers
    text = re.sub(r'[0-9]', '', text)
    
    #remove punctuations except ! and ?
    text = re.sub(r'[^\w\s!?]', '', text)
    
    #ronvert to lowercase
    text = text.lower()
    
    # tokenize text
    text_tokens = word_tokenize(text)

    #remove stopwords but keep 'not' and 'no'
    stop_words = set(stopwords.words('english'))
    stop_words.discard('no')  # Retain 'no'
    stop_words.discard('not')  # Retain 'not'
    
    text_tokens = [word for word in text_tokens if word not in stop_words]
    
    #POS tagging
    pos_tags = pos_tag(text_tokens)
    
    #lemmatize based on POS and remove stopwords
    text = [
        lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos)) 
        for word, pos in pos_tags 
        if len(word) > 1  # Ensure word length is >1 (remove single characters)
    ]
    
    # Return the cleaned text
    return ' '.join(text)

# Example use case
text = "I can't do this anymore. I'm running away now, but not without a reason. He was acting quickly."
cleaned_text = clean_data(text)
print(cleaned_text)

not anymore run away not without reason act quickly


In [35]:
lemmatizer.lemmatize("strongly",pos="r")

'strongly'

In [36]:
test = "I can't do this anymore 1 <head> ?? I am running away now"
clean_data(test)

'cannot anymore ?? run away'

In [14]:
data = "The cats are running quickly in the park!"
cleaned_data = clean_data(data)
print(cleaned_data)

cat run quickly park
